In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import xgboost as xgb

In [24]:
data=pd.read_csv('drive/MyDrive/class.csv', encoding='utf-8')

In [25]:
###转换格式
###转换日期
month_dict = {
    '1月': 'January', '2月': 'February', '3月': 'March',
    '4月': 'April', '5月': 'May', '6月': 'June',
    '7月': 'July', '8月': 'August', '9月': 'September',
    '10月': 'October', '11月': 'November', '12月': 'December'
}
def convert_to_english_date(chinese_date_str):
    month_chinese = chinese_date_str.split('月')[0] + '月'
    year = chinese_date_str.split('月')[1]
    month_english = month_dict.get(month_chinese, 'Unknown')
    return f"{month_english} {year}"

data['Year.Month'] = data['Year.Month'].apply(convert_to_english_date)
data['Year.Month'] = pd.to_datetime(data['Year.Month'], errors='coerce')
data.rename(columns={'Year.Month': 'Date'}, inplace=True)
data.sort_values(by='Date', inplace=True)

In [27]:
###转换年龄
def classify_age_group(age_group):
    if age_group in ['0 - 4 years inclusive', '5 - 9 years inclusive', '10 - 14 years inclusive', '15 - 19 years inclusive']:
        return 'Teenagers'
    elif age_group in ['20 - 24 years inclusive', '25 - 29 years inclusive', '30 - 34 years inclusive', '35 - 39 years inclusive']:
        return 'Middle-aged-young'
    elif age_group in ['40 - 44 years inclusive', '45 - 49 years inclusive', '50 - 54 years inclusive', '55 - 59 years inclusive']:
        return 'Middle-aged-old'
    elif age_group in ['60 - 64 years inclusive', '65 - 69 years inclusive', '70 - 74 years inclusive', '75 - 79 years inclusive', '80 years or over']:
        return 'Elderly'
    else:
        return 'Unknown'
data['Age.Group.5Yr.Band'] = data['Age.Group.5Yr.Band'].apply(classify_age_group)

In [35]:
data = data[data['Ethnic.Group'] != 'Not Stated']
data = data[data['SEX'] != 'Not Stated']

In [41]:
data

,Date,Victimisations,SEX,Age.Group.5Yr.Band,OOI.Exclusion,Ethnic.Group,ANZSOC.Group,class
18395,2015-06-01,2,Female,Middle-aged-young,Court action,Maori,"Theft (Except Motor Vehicles), N.E.C.",0
23448,2015-06-01,1,Male,Middle-aged-young,Court action,European,"Theft (Except Motor Vehicles), N.E.C.",0
62710,2015-06-01,2,Male,Middle-aged-old,Court action,Pacific Island,Serious Assault Resulting in Injury,0
66759,2015-06-01,1,Male,Elderly,Court action,European,Illegal Use of a Motor Vehicle,0
23211,2015-06-01,2,Male,Middle-aged-young,Court action,European,Aggravated Robbery,0
...,...,...,...,...,...,...,...,...
70614,2023-07-01,1,Female,Middle-aged-young,Court action,Pacific Island,Illegal Use of a Motor Vehicle,0
70613,2023-07-01,2,Female,Middle-aged-young,Court action,Pacific Island,Illegal Use of a Motor Vehicle,0
71770,2023-07-01,1,Female,Middle-aged-old,Court action,Asian,Abduction and Kidnapping,0
71771,2023-07-01,1,Female,Middle-aged-young,Court action,European,Abduction and Kidnapping,0


In [43]:
label_cols = ['SEX', 'Age.Group.5Yr.Band', 'Ethnic.Group', 'ANZSOC.Group']
for col in label_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

In [45]:
X = data.drop(columns=['Date', 'class' ,'OOI.Exclusion'])
y = data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [97]:
clf = xgb.XGBClassifier(n_estimators=50, learning_rate=0.1,max_depth=10,subsample=0.1, colsample_bytree=0.8)
clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [98]:
y_pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))

Accuracy: 0.8798344739773993
